In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Train Incrementally and Evaluate



In [116]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from torch.utils.data import DataLoader, TensorDataset

class InitialColumnProgNN(nn.Module):
    def __init__(self, topology, activations, lr):
        super(InitialColumnProgNN, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(len(topology) - 1):
            self.layers.append(nn.Linear(topology[i], topology[i+1]))
        self.activations = activations

        # optimizer and criterion
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr=lr)

    def forward(self, x):
        for i, layer in enumerate(self.layers):
            x = layer(x)
            # removinf softmax from last layer because of crossentropyloss function - vishal
            if i != len(self.activations):
              x = self.activations[i](x)
        return x

class ExtensibleColumnProgNN(nn.Module):
    def __init__(self, topology, activations, prev_columns, lr):
        super(ExtensibleColumnProgNN, self).__init__()
        self.layers = nn.ModuleList()
        self.lateral_connections = nn.ModuleList()
        for i in range(len(topology) - 1):
            self.layers.append(nn.Linear(topology[i], topology[i+1]))
            if i > 0:
                lateral = [nn.Linear(prev_column.layers[i-1].out_features, topology[i+1], bias=False) for prev_column in prev_columns]
                self.lateral_connections.append(nn.ModuleList(lateral))
        self.activations = activations
        self.prev_columns = prev_columns

        # optimizer and criterion
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr=lr)

    def forward(self, x):
        prev_hs = [[[] for j in range(len(prev_col.layers))] for prev_col in self.prev_columns]

        for j in range(len(self.prev_columns)):
            x_copy = x.clone()
            for i, col_layer in enumerate(self.prev_columns[j].layers):
                x_copy = col_layer(x_copy)
                # removinf softmax from last layer because of crossentropyloss function - vishal
                if i != len(self.prev_columns[j].activations):
                  x_copy = self.prev_columns[j].activations[i](x_copy)
                prev_hs[j][i] = x_copy.clone()


        for i, layer in enumerate(self.layers):
            x = layer(x)
            if i > 0:
                for j, lateral in enumerate(self.lateral_connections[i-1]):
                    x += lateral(prev_hs[j][i - 1])
            # removinf softmax from last layer because of crossentropyloss function - vishal
            if i != len(self.activations):
              x = self.activations[i](x)
        return x

#todo: add training batch size later
def train_column(column, data, target, epochs=50, batch_size=32):
    # Create a dataset and data loader for batching
    dataset = TensorDataset(data, target)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # saved h_values for training lateral connections
    for epoch in range(epochs):
        for batch_data, batch_target in data_loader:
            column.optimizer.zero_grad()

            # forward pass
            output = column(batch_data)

            loss = column.criterion(output, batch_target)
            loss.backward()
            column.optimizer.step()
        if epoch%10 == 9:
          print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}')

def test_column(column, embeddings, labels):
    column.eval()  # Set the model to evaluation mode todo: do we need to unset eval mode after testing??
    with torch.no_grad():
        test_outputs = column(embeddings)
        logits, predicted = torch.max(test_outputs, 1)
        return (logits, predicted)

        # # Calculate accuracy
        # accuracy = accuracy_score(labels, predicted)

        # # Calculate F1 score
        # f1 = f1_score(labels, predicted)

        # # Calculate precision
        # precision = precision_score(labels, predicted)

        # # Calculate recall
        # recall = recall_score(labels, predicted)

        # print("Accuracy: {:.2f}".format(accuracy))
        # print("F1 Score: {:.2f}".format(f1))
        # print("Precision: {:.2f}".format(precision))
        # print("Recall: {:.2f}".format(recall))

class PNN():
    def __init__(self):
        self.num_classes = 1

        self.topology = [256, 100, 64, 25, 2]
        self.activations = [F.relu, F.relu, F.relu]

        self.lr = 0.001

        # Instantiate the first module
        self.subnetworks = [InitialColumnProgNN(self.topology, self.activations, self.lr)]

        # # Define the layer on top
        # self.output_layer = nn.Linear(in_features=num_classes*2, out_features=num_classes)

    def add_network(self):
        self.num_classes += 1
        self.subnetworks.append(ExtensibleColumnProgNN(self.topology, self.activations, self.subnetworks, self.lr))

    def train(self, subnetwork, embeddings, labels):
        for i in range(self.num_classes):
            if i == subnetwork:
                # Unfreeze the parameters of the modules
                for param in self.subnetworks[i].parameters():
                    param.requires_grad = True
            else:
                # Freeze the parameters of the modules
                for param in self.subnetworks[i].parameters():
                    param.requires_grad = False
        # Train the relevant PNN
        train_column(self.subnetworks[subnetwork], embeddings, labels, epochs=50)

        #Unfreeze all parameters
        for i in range(self.num_classes):
            for param in self.subnetworks[i].parameters():
                    param.requires_grad = True

    def test(self, embeddings, labels, i):
        confidence_scores_class_arr = []
        predictions_class_arr = []
        for i in range(self.num_classes):
            # print('\nResults for class', i, '-')
            confidence_scores, predictions = test_column(self.subnetworks[i], embeddings, labels[i])
            confidence_scores_class_arr.append(confidence_scores)
            predictions_class_arr.append(predictions)

        one_hot_predictions = []
        for j in range(len(labels)):
            tmp = [0]*self.num_classes

            max_confidence = float('-inf')
            max_idx = -1
            for i in range(self.num_classes):
                if confidence_scores_class_arr[i][j] > max_confidence and predictions_class_arr[i][j] == 1:
                    max_confidence = confidence_scores_class_arr[i][j]
                    max_idx = i

            if max_idx != -1:
                tmp[max_idx] = 1

            one_hot_predictions.append(tmp)

        # Convert one-hot encoded ground truth and predictions to class labels
        true_class_labels = []
        predicted_class_labels = []
        for j in range(len(labels)):
            flag = True
            for i in range(self.num_classes):
                if labels[j][i] == 1:
                    flag = False
                    true_class_labels.append(i + 1)
                    break
            if flag:
                true_class_labels.append(0)

        for j in range(len(one_hot_predictions)):
            flag = True
            for i in range(self.num_classes):
                if one_hot_predictions[j][i] == 1:
                    flag = False
                    predicted_class_labels.append(i + 1)
                    break
            if flag:
                predicted_class_labels.append(0)

        print(true_class_labels)
        print(predicted_class_labels)

        # Compute metrics
        accuracy = accuracy_score(true_class_labels, predicted_class_labels)
        f1 = f1_score(true_class_labels, predicted_class_labels, average='macro')
        precision = precision_score(true_class_labels, predicted_class_labels, average='macro')
        recall = recall_score(true_class_labels, predicted_class_labels, average='macro')

        print(f'Accuracy: {accuracy}')
        print(f'F1 Score: {f1}')
        print(f'Precision: {precision}')
        print(f'Recall: {recall}')


model = PNN()

# train_column(model, embeddings0, labels0, epochs=50)

In [117]:
model.subnetworks

[InitialColumnProgNN(
   (layers): ModuleList(
     (0): Linear(in_features=256, out_features=100, bias=True)
     (1): Linear(in_features=100, out_features=64, bias=True)
     (2): Linear(in_features=64, out_features=25, bias=True)
     (3): Linear(in_features=25, out_features=2, bias=True)
   )
   (criterion): CrossEntropyLoss()
 )]

# Task 1

In [118]:
import json
import torch
embeddings = []
labels = []

for i in range(1, 5):
    with open(f'/content/drive/MyDrive/CSCI544-Project/data/class_wise_embeddings/train_embeddings/dos_train_file_{i}.json', 'r') as f:
        for line in f:
            data = json.loads(line)
            embeddings.append(data['embeddings'][0])
            labels.append(data['label'])

# Convert lists to tensors
embeddings0 = torch.tensor(embeddings)
labels0 = torch.tensor(labels)

print(embeddings0.size())  # This will show the shape of the embeddings tensor
print(labels0.size())     # This will show the shape of the labels tensor

model.train(0, embeddings=embeddings0, labels=labels0)

torch.Size([4437, 256])
torch.Size([4437])
Epoch 10/50, Loss: 0.4558800756931305
Epoch 20/50, Loss: 0.019321078434586525
Epoch 30/50, Loss: 0.021809855476021767
Epoch 40/50, Loss: 0.002580881817266345
Epoch 50/50, Loss: 0.020021792501211166


In [119]:
embeddings = []
labels = []

with open(f'/content/drive/MyDrive/CSCI544-Project/data/class_wise_embeddings/test_embeddings/dos_test_file.json', 'r') as f:
    for line in f:
        data = json.loads(line)
        embeddings.append(data['embeddings'][0])
        labels.append([data['label']])

# Convert lists to tensors
embeddings0 = torch.tensor(embeddings)
labels0 = torch.tensor(labels)

print(embeddings0.size())  # This will show the shape of the embeddings tensor
print(labels0.size())     # This will show the shape of the labels tensor

model.test(embeddings0, labels0, 0)

torch.Size([1109, 256])
torch.Size([1109, 1])
[0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,

# Task 2

In [120]:
import json
import torch
embeddings = []
labels = []

for i in range(1, 5):
    with open(f'/content/drive/MyDrive/CSCI544-Project/data/class_wise_embeddings/train_embeddings/+info_train_file_{i}.json', 'r') as f:
        for line in f:
            data = json.loads(line)
            embeddings.append(data['embeddings'][0])
            labels.append(data['label'])

# Convert lists to tensors
embeddings0 = torch.tensor(embeddings)
labels0 = torch.tensor(labels)

print(embeddings0.size())  # This will show the shape of the embeddings tensor
print(labels0.size())     # This will show the shape of the labels tensor

model.add_network()
model.train(1, embeddings=embeddings0, labels=labels0)

torch.Size([984, 256])
torch.Size([984])
Epoch 10/50, Loss: 0.6026760935783386
Epoch 20/50, Loss: 0.00683538056910038
Epoch 30/50, Loss: 0.01303717028349638
Epoch 40/50, Loss: 0.059830281883478165
Epoch 50/50, Loss: 0.001667840057052672


In [121]:
embeddings = []
labels = []

with open(f'/content/drive/MyDrive/CSCI544-Project/data/class_wise_embeddings/test_embeddings/dos_test_file.json', 'r') as f:
    for line in f:
        data = json.loads(line)
        embeddings.append(data['embeddings'][0])
        labels.append([data['label'], 0])

with open(f'/content/drive/MyDrive/CSCI544-Project/data/class_wise_embeddings/test_embeddings/+info_test_file.json', 'r') as f:
    for line in f:
        data = json.loads(line)
        embeddings.append(data['embeddings'][0])
        labels.append([0, data['label']])

# Convert lists to tensors
embeddings0 = torch.tensor(embeddings)
labels0 = torch.tensor(labels)

model.test(embeddings0, labels0, 1)

[0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 

# Task 3

In [122]:
import json
import torch
embeddings = []
labels = []

for i in range(1, 5):
    with open(f'/content/drive/MyDrive/CSCI544-Project/data/class_wise_embeddings/train_embeddings/bypass_train_file_{i}.json', 'r') as f:
        for line in f:
            data = json.loads(line)
            embeddings.append(data['embeddings'][0])
            labels.append(data['label'])

# Convert lists to tensors
embeddings0 = torch.tensor(embeddings)
labels0 = torch.tensor(labels)

print(embeddings0.size())  # This will show the shape of the embeddings tensor
print(labels0.size())     # This will show the shape of the labels tensor

model.add_network()
model.train(2, embeddings=embeddings0, labels=labels0)

embeddings = []
labels = []

with open(f'/content/drive/MyDrive/CSCI544-Project/data/class_wise_embeddings/test_embeddings/dos_test_file.json', 'r') as f:
    for line in f:
        data = json.loads(line)
        embeddings.append(data['embeddings'][0])
        labels.append([data['label'], 0, 0])

with open(f'/content/drive/MyDrive/CSCI544-Project/data/class_wise_embeddings/test_embeddings/+info_test_file.json', 'r') as f:
    for line in f:
        data = json.loads(line)
        embeddings.append(data['embeddings'][0])
        labels.append([0, data['label'], 0])

with open(f'/content/drive/MyDrive/CSCI544-Project/data/class_wise_embeddings/test_embeddings/bypass_test_file.json', 'r') as f:
    for line in f:
        data = json.loads(line)
        embeddings.append(data['embeddings'][0])
        labels.append([0, 0, data['label']])

# Convert lists to tensors
embeddings0 = torch.tensor(embeddings)
labels0 = torch.tensor(labels)

model.test(embeddings0, labels0, 2)

torch.Size([709, 256])
torch.Size([709])
Epoch 10/50, Loss: 0.4395773410797119
Epoch 20/50, Loss: 0.024652214720845222
Epoch 30/50, Loss: 0.01570112816989422
Epoch 40/50, Loss: 0.00047700299182906747
Epoch 50/50, Loss: 0.0005028137238696218
[0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1,

# Task 4

In [123]:
import json
import torch
embeddings = []
labels = []

for i in range(1, 5):
    with open(f'/content/drive/MyDrive/CSCI544-Project/data/class_wise_embeddings/train_embeddings/+priv_train_file_{i}.json', 'r') as f:
        for line in f:
            data = json.loads(line)
            embeddings.append(data['embeddings'][0])
            labels.append(data['label'])

# Convert lists to tensors
embeddings0 = torch.tensor(embeddings)
labels0 = torch.tensor(labels)

print(embeddings0.size())  # This will show the shape of the embeddings tensor
print(labels0.size())     # This will show the shape of the labels tensor

model.add_network()
model.train(3, embeddings=embeddings0, labels=labels0)

embeddings = []
labels = []

with open(f'/content/drive/MyDrive/CSCI544-Project/data/class_wise_embeddings/test_embeddings/dos_test_file.json', 'r') as f:
    for line in f:
        data = json.loads(line)
        embeddings.append(data['embeddings'][0])
        labels.append([data['label'], 0, 0, 0])

with open(f'/content/drive/MyDrive/CSCI544-Project/data/class_wise_embeddings/test_embeddings/+info_test_file.json', 'r') as f:
    for line in f:
        data = json.loads(line)
        embeddings.append(data['embeddings'][0])
        labels.append([0, data['label'], 0, 0])

with open(f'/content/drive/MyDrive/CSCI544-Project/data/class_wise_embeddings/test_embeddings/bypass_test_file.json', 'r') as f:
    for line in f:
        data = json.loads(line)
        embeddings.append(data['embeddings'][0])
        labels.append([0, 0, data['label'], 0])

with open(f'/content/drive/MyDrive/CSCI544-Project/data/class_wise_embeddings/test_embeddings/+priv_test_file.json', 'r') as f:
    for line in f:
        data = json.loads(line)
        embeddings.append(data['embeddings'][0])
        labels.append([0, 0, 0, data['label']])

# Convert lists to tensors
embeddings0 = torch.tensor(embeddings)
labels0 = torch.tensor(labels)

model.test(embeddings0, labels0, 3)

torch.Size([638, 256])
torch.Size([638])
Epoch 10/50, Loss: 0.4209142327308655
Epoch 20/50, Loss: 0.05126958712935448
Epoch 30/50, Loss: 0.0052050333470106125
Epoch 40/50, Loss: 0.002332912525162101
Epoch 50/50, Loss: 0.002376549644395709
[0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1

# Task 5

In [124]:
import json
import torch
embeddings = []
labels = []

for i in range(1, 5):
    with open(f'/content/drive/MyDrive/CSCI544-Project/data/class_wise_embeddings/train_embeddings/other_train_file_{i}.json', 'r') as f:
        for line in f:
            data = json.loads(line)
            embeddings.append(data['embeddings'][0])
            labels.append(data['label'])

# Convert lists to tensors
embeddings0 = torch.tensor(embeddings)
labels0 = torch.tensor(labels)

print(embeddings0.size())  # This will show the shape of the embeddings tensor
print(labels0.size())     # This will show the shape of the labels tensor

model.add_network()
model.train(4, embeddings=embeddings0, labels=labels0)

embeddings = []
labels = []

with open(f'/content/drive/MyDrive/CSCI544-Project/data/class_wise_embeddings/test_embeddings/dos_test_file.json', 'r') as f:
    for line in f:
        data = json.loads(line)
        embeddings.append(data['embeddings'][0])
        labels.append([data['label'], 0, 0, 0, 0])

with open(f'/content/drive/MyDrive/CSCI544-Project/data/class_wise_embeddings/test_embeddings/+info_test_file.json', 'r') as f:
    for line in f:
        data = json.loads(line)
        embeddings.append(data['embeddings'][0])
        labels.append([0, data['label'], 0, 0, 0])

with open(f'/content/drive/MyDrive/CSCI544-Project/data/class_wise_embeddings/test_embeddings/bypass_test_file.json', 'r') as f:
    for line in f:
        data = json.loads(line)
        embeddings.append(data['embeddings'][0])
        labels.append([0, 0, data['label'], 0, 0])

with open(f'/content/drive/MyDrive/CSCI544-Project/data/class_wise_embeddings/test_embeddings/+priv_test_file.json', 'r') as f:
    for line in f:
        data = json.loads(line)
        embeddings.append(data['embeddings'][0])
        labels.append([0, 0, 0, data['label'], 0])

with open(f'/content/drive/MyDrive/CSCI544-Project/data/class_wise_embeddings/test_embeddings/other_test_file.json', 'r') as f:
    for line in f:
        data = json.loads(line)
        embeddings.append(data['embeddings'][0])
        labels.append([0, 0, 0, 0, data['label']])

# Convert lists to tensors
embeddings0 = torch.tensor(embeddings)
labels0 = torch.tensor(labels)

model.test(embeddings0, labels0, 4)

torch.Size([800, 256])
torch.Size([800])
Epoch 10/50, Loss: 0.23601451516151428
Epoch 20/50, Loss: 0.0087825246155262
Epoch 30/50, Loss: 0.0010724071180447936
Epoch 40/50, Loss: 0.0003146614180877805
Epoch 50/50, Loss: 0.03665764629840851
[0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1